In [ ]:
# We are creating a classifier to predict whether a person is touching grass or not
# We are using the data from a temp and humidity sensor and a uv sensor to predict whether a person is touching grass or not

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# use GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# Importing the dataset
dataset = json.load(open('train.json'))

In [ ]:
df = pd.DataFrame(dataset)

df['class'] = df['class'].map({'Touching Grass': 1, 'Not Touching Grass': 0})

X = df.iloc[:, :-1].values
y = df.iloc[:, 3].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
mean = accuracies.mean()
std = accuracies.std()

parameters = [{'learning_rate': [0.1, 0.01, 0.001], 'n_estimators': [100, 200, 300]}]
grid_search = GridSearchCV(estimator = classifier,
                            param_grid = parameters,
                            scoring = 'accuracy',
                            cv = 10,
                            n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(best_accuracy)
print(best_parameters)
new_classifier = GradientBoostingClassifier(learning_rate = best_parameters['learning_rate'], n_estimators = best_parameters['n_estimators'])
# train the new classifier
new_classifier.fit(X_train, y_train)
# predict the test set results
new_y_pred = new_classifier.predict(X_test)
print(new_classifier.score(X_test, y_test))



In [ ]:
# Now we need to export the classifier to a file so that we can use it for iOS
import coremltools
coreml_model = coremltools.converters.sklearn.convert(new_classifier, ['temp', 'humidity', 'uv'], 'class')
coreml_model.save('tuchgrass.mlmodel')